In [1]:
from vizproo import CustomWidget, MatrixLayout, MatrixCreator
import traitlets
import pandas as pd
import numpy as np

In [2]:
#d3.select(element).append(() => svg.node());

# Custom Chart
Nuestra libreria nos permite crear gráficos personalizados mediante la extensión de la clase "CustomWidget". A continuación, se explica cómo crear un gráfico personalizado paso a paso.

## Ejemplo básico
Para poder crear un gráfico personalizado, es necesario entender la estructura básica de la clase "CustomWidget". Esta clase será la base para crear nuestro gráfico.

```python
class MyChart(CustomWidget):
    _esm = "my_chart.js"
    _css = "my_chart.css" #Opcional
```    
- `_esm`: Este atributo define el archivo JavaScript que contiene la lógica del gráfico. Debe ser un archivo válido que implemente las funcionalidades necesarias para renderizar el gráfico. En celdas posteriores se explicará lo mínimo necesario para crear este archivo.
- `_css`: Este atributo es opcional y define el archivo CSS que contiene los estilos personalizados para el gráfico. Si no se proporciona, el gráfico utilizará estilos predeterminados.

In [2]:
path = "../samples/line.js"
csspath = "../samples/line_2.css" #Linea negra. Con line.css es linea blanca
class CustomLeo(CustomWidget):
    _esm = CustomWidget.createWidgetFromLocalFile(paramList=[], filePath=path)
    _css = csspath

line = CustomLeo()
line

## Grafico que acepte diferentes datos
A continuación, se muestra un ejemplo de un gráfico personalizado que acepta diferentes datos y los renderiza.
*Nota*: Aveces la data de un gráfico se procesa en el mismo grafico pero muchas veces se procesa por fuera y se le pasa al gráfico ya procesada. En nuestro caso al ser "Custom" es necesario que te asegures que la data que normalmente la trabajabas como "javascript" tenga la estructura correcta en python.


### Este es un ejemplo de  funciones para preprocesar un diccionario de datos en python para luego pasarselo a un gráfico custom. Por ejemplo javascript no soporta tipos de datos numpy como "-inf" o "nan" por lo que es necesario convertirlos a tipos de datos que javascript pueda entender como "null".

In [3]:
def read_csv_to_dicts(file_path: str) -> list[dict]:
    df = pd.read_csv(file_path)
    # Sustituye NaN, inf, -inf por None (null en JS)
    df = df.replace({np.nan: None, np.inf: None, -np.inf: None})
    return df.to_dict('records')
def get_columns(data:list[dict]) -> list[str]:
    return list(data[0].keys()) if data else []

### Dataset

In [4]:
penguins = read_csv_to_dicts("../samples/penguins.csv")

In [5]:
display(penguins[0])

{'species': 'Adelie',
 'island': 'Torgersen',
 'culmen_length_mm': 39.1,
 'culmen_depth_mm': 18.7,
 'flipper_length_mm': 181.0,
 'body_mass_g': 3750.0,
 'sex': 'MALE'}

### Crear atributos de la clase CustomWidget
Puedes agregar atributos al gráfico personalizado para manejar diferentes configuraciones o datos. Aquí hay un ejemplo de cómo hacerlo:

```python
class MyChart(CustomWidget):
    _esm = "my_chart.js"
    _css = "my_chart.css"

    data = traitlets.List([]).tag(sync=True) # Atributo para almacenar los datos del gráfico
```
Es necesario usar "traitlets" para definir los atributos de la clase que se sincronizarán entre Python y JavaScript. En este caso, hemos definido un atributo "data" que es una lista y se sincroniza automáticamente.


In [12]:
path = "../samples/scatterplot_matrix.js"

class CustomScatter(CustomWidget):
    _esm = CustomWidget.createWidgetFromLocalFile(paramList=["data"],
                                                  height = 1100,
                                                filePath=path)
    
    data = traitlets.List([]).tag(sync=True)

chart = CustomScatter(data=penguins)
chart

## Python <---> JavaScript
Tienes la capacidad de interactuar con el gráfico personalizado de tal forma que al seleccionar algún valor de este puedas obtener ese valor en python. Para esto es necesario definir un atributo en la clase "CustomWidget" y luego desde javascript actualizar ese atributo cuando se seleccione algún valor.
*Extra imports*: Es importante que si tienes imports adicionales en el archivo javascript los agregues en el atributo "extra_imports" de la función "createWidgetFromLocalFile".
*Height*: Puedes definir la altura del gráfico personalizado desde python usando el atributo "height" de la función "createWidgetFromLocalFile".
```python
path = "../samples/mapa.js"
mapa_css = "../samples/mapa.css"
class CustomMapa(CustomWidget):
    _esm = CustomWidget.createWidgetFromLocalFile(paramList=["data"], filePath=path,
                                                  height = 610,
                                                  extra_imports=['import * as topojson from "https://esm.sh/topojson-client@3";'])
    _css = mapa_css
    data = traitlets.Dict(mapa_data).tag(sync=True)
    selectedValues = traitlets.List([]).tag(sync=True)
```

In [13]:
import json
path_ = "../samples/states-albers-10m.json"
mapa_data = json.load(open(path_))

In [18]:
path = "../samples/mapa.js"
mapa_css = "../samples/mapa.css"
class CustomMapa(CustomWidget):
    _esm = CustomWidget.createWidgetFromLocalFile(paramList=["data"], filePath=path,
                                                  height = 610,
                                                  extra_imports=['import * as topojson from "https://esm.sh/topojson-client@3";'])
    _css = mapa_css
    data = traitlets.Dict(mapa_data).tag(sync=True)
    selectedValues = traitlets.List([]).tag(sync=True)

    def on_select_values(self, callback):
        self.observe(callback, names=["selectedValues"])
    

mapa = CustomMapa(data = mapa_data)

In [19]:
m = MatrixCreator(7,4)
m

MatrixCreator(columns='4', rows='7')

In [ ]:
# Para poder hacer la linea blanca usa "leo.css"

In [20]:
layout = MatrixLayout(m.matrix,style='dark')
layout.add(line,1)
layout.add(mapa,2)
layout

MatrixLayout(grid_areas=['slpOtZgrAF', 'bBTbyMJXCs'], grid_template_areas='\n"slpOtZgrAF slpOtZgrAF slpOtZgrAF…

In [ ]:
mapa.selectedValues